# Price prediction from Airbnb Listing Data
### Import libraries  

In [21]:
import pandas as pd
import os

### Read the Data and Pick the attributes

In [22]:
files = [file for file in os.listdir('./Listing Data After COVID')]
neighbourhoods_df = pd.read_csv('neighbourhoods.csv')

after_COVID_df = pd.DataFrame()

for file in files:
    df = pd.read_csv("./Listing Data After COVID/"+ file)
    df = df[df.room_type.eq("Entire home/apt")]
    df = df[['id', 'neighbourhood', 'neighbourhood_cleansed', 
             'zipcode', 'latitude', 'longitude', 'price']]
    after_COVID_df = pd.concat([after_COVID_df, df])

after_COVID_df.to_csv("after_COVID.csv", index=False)
print("File created")
after_COVID_df.head()

c:\users\ss042\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (61,62,94) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\ss042\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (43,61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\ss042\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (61,62,94,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


File created


,id,neighbourhood,neighbourhood_cleansed,zipcode,latitude,longitude,price
1,2595,Midtown,Midtown,10018,40.75362,-73.98377,$225.00
2,3831,Brooklyn,Clinton Hill,11238,40.68514,-73.95976,$89.00
3,5099,Manhattan,Murray Hill,10016,40.74767,-73.97500,$200.00
8,5238,Lower East Side,Chinatown,10002,40.71344,-73.99037,$150.00
10,5552,Manhattan,West Village,10014,40.73552,-74.01042,$160.00


### Encode a caegorical data into integer

In [24]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
neighbourhoods_df['neighbourhood_group_number'] = lb_make.fit_transform(neighbourhoods_df['neighbourhood_group'])
neighbourhoods_df['neighbourhood_number'] = lb_make.fit_transform(neighbourhoods_df['neighbourhood'])

### Merge the data with neighbourhood table

In [25]:
Listing_neighbourhood = pd.merge(after_COVID_df, neighbourhoods_df, 
         left_on= "neighbourhood_cleansed", right_on="neighbourhood", 
         how = 'left')

### Create the new columns that represents the neighbourhood and its group

In [26]:
after_COVID_df['neighbourhood_group_number'] = Listing_neighbourhood['neighbourhood_group_number']
after_COVID_df['neighbourhood_number'] = Listing_neighbourhood['neighbourhood_number']
after_COVID_df = after_COVID_df.drop(columns = ['neighbourhood', 'neighbourhood_cleansed'], axis=1)

### Check the missing rows and take care of it

In [27]:
nan_df = after_COVID_df[after_COVID_df.isna().any(axis =1 )]
print(nan_df.count())

id                            1202
zipcode                          0
latitude                      1202
longitude                     1202
price                         1202
neighbourhood_group_number    1202
neighbourhood_number          1202
dtype: int64


### Delete the dublications

In [28]:
duplicate_df = after_COVID_df[after_COVID_df['id'].duplicated(keep=False)]
result = duplicate_df.sort_values(by=['id'])
after_COVID_df.drop_duplicates(subset ="id", keep = False, inplace = True) 
after_COVID_df.head(10)

,id,zipcode,latitude,longitude,price,neighbourhood_group_number,neighbourhood_number
86,22983,10002,40.71938,-73.99101,$300.00,2,218
734,287481,11206,40.70839,-73.94289,$120.00,2,40
1330,657734,11233,40.68248,-73.92246,$250.00,3,4
1380,686480,10031,40.82379,-73.94849,$150.00,1,44
1480,743150,10040,40.85410,-73.93365,$125.00,2,92
1767,853549,11217,40.67169,-73.97436,$178.00,1,55
1962,1020625,10075,40.77432,-73.95787,$136.00,2,218
2003,1062983,10002,40.71809,-73.99051,$68.00,2,210
2039,1095278,10301,40.63324,-74.09678,$143.00,1,223
2465,1573073,10036,40.75889,-73.99054,$150.00,1,225


### Apply the Classifier method for neighbourhood group